<a href="https://colab.research.google.com/github/LennyMaina/Customer_behaviour_analysis/blob/main/FInance_Act_RAG_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Embedding

In [24]:
pip install sentence-transformers numpy chromadb python-dotenv

In [11]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('BAAI/bge-small-en-v1.5')

In [14]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.2/331.2 kB 8.4 MB/s eta 0:00:00


In [15]:
from pypdf import PdfReader

In [70]:
def extract_text(path):
    reader = PdfReader(path)
    text = ''
    for page in reader.pages:
      text += page.extract_text() + '\n'
    return text

In [ ]:
text = extract_text('/content/The-Finance-Act--2023.pdf')

In [75]:
words = text.split()

In [76]:
len(words)

14611

In [84]:
def chunk_text(text, size = 250):
  words = text.split()
  chunks = []
  for i in range(0,len(words),size):
    chunks.append(' '.join(words[i:i+size]))
  return chunks

In [85]:
chunks = chunk_text(text)

In [87]:
act_emb = model.encode(chunks).tolist()

In [88]:
from numpy import dot

In [90]:
from numpy.linalg import norm

Storing in a database (RAG)

In [91]:
import chromadb

In [92]:
client = chromadb.PersistentClient(path = './chromadb')

In [96]:
collection = client.get_or_create_collection(
    name = 'FinanceAct',
    metadata = {'description': 'My Finance Act collection'}

)
print(collection.name)

FinanceAct


In [102]:
ids = [f'doc_{i}' for i in range(len(chunks))]

In [103]:
collection.add(
    documents = chunks,
    embeddings = act_emb,
    ids = ids
)

In [104]:
collection.count()

59

In [105]:
query = 'What does the Finance Act say about Finance'

In [114]:
query_embedding = model.encode([query]).tolist()

In [115]:
answer = collection.query(
    query_embeddings = query_embedding,
    n_results = 1
)

In [116]:
answer

{'ids': [['doc_0']],
 'embeddings': None,
 'documents': [['SPECIAL ISSUE Kenya Gazette Supplement No. 97 (Acts No. 4) REPUBLIC OF KENYA –––––––"KENYA GAZETTE SUPPLEMENT ACTS, 2023 NAIROBI, 26th June, 2023 CONTENT Act— PAGE The Finance Act, 2023 ..................................................................................... 79 PRINTED AND PUBLISHED BY THE GOVERNMENT PRINTER, NAIROBI 79 THE FINANCE ACT, 2023 No. 4 of 2023 Date of Assent: 26th June, 2023 Date of Commencement: See Section 1 AN ACT of Parliament to amend the laws relating to various taxes and duties; and for matters incidental thereto ENACTED by the Parliament of Kenya, as follows – PART I—PRELIMINARY 1. This Act may be cited as the Finance Act, 2023, and shall come into operation or be deemed to have come into operation as follows— Short title and commencement. (a)!on the 1st of September, 2023, sections 10, 26(b)(xiii), 52, 56, 63, 64 and 74; (b)!on the 1st January, 2024, sections 5(c), 6, 12, 14, 20, 25, 26(a), 26(


A similarity score measures how much two pieces of data (like words or images) resemble each other by comparing their mathematical "coordinates."

Why Lower Distance = Better Match
Think of it like physical proximity. If you’re standing right next to someone, the distance between you is nearly zero. If you’re on the other side of town, the distance is high.

In data science:

Distance measures the gap between two points.

0 Distance means the items are identical (they occupy the exact same spot).

Higher Distance means the items are increasingly different.

Cosine Similarity vs. Distance
Cosine similarity measures the angle between two vectors.

If two vectors point in the same direction, the angle is 0
∘
  and the similarity is 1.

To get Cosine Distance, we use the formula: 1−Similarity.

Therefore, a Similarity of 1 results in a Distance of 0. When you are looking for the best match, you are looking for the smallest possible gap between your data points.